# Model 3
- add more fetures and replace na's
- ada boosting, xg boosting, 
- pick two best models
- import data from csv
- voting classifier with different weights(maybe different weights list function) and grid search
- add more testing data

## Import Libraries

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn.metrics import mean_absolute_error, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, ExtraTreesClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

import pandas as pd
import numpy as np


## Import Subreddit DF's

In [2]:
df = pd.read_csv('../data/Subreddits_Data.csv')

In [3]:
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [4]:
df[['selftext', 'title', 'corrected_title']] = df[['selftext', 'title', 'corrected_title']].fillna('')

In [5]:
df.isna().sum()

subreddit          0
selftext           0
title              0
corrected_title    0
char_count         0
word_count         0
neg                0
neu                0
pos                0
compound           0
dtype: int64

## Count Vectorize

In [6]:
import nltk
from nltk.corpus import stopwords
stop_words  = stopwords.words('english')
cv = CountVectorizer(stop_words=stop_words, ngram_range=(1,1))

cv.fit(df['corrected_title'])
df_cv = pd.DataFrame(cv.transform(df['corrected_title']).A, columns=cv.get_feature_names_out())
df_cv


,___,aaa,aaaa,aaaaaa,aak,aave,ab,abandonment,abdomen,abilities,...,zombies,zone,zones,zoo,zoom,zooming,zoos,zorro,zuckerberg,zzquil
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5655,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# df_cv.drop(columns=['subreddit', 'title'], axis=1, inplace = True)

In [10]:
master_df = pd.concat([df, df_cv], axis=1)
master_df = master_df.loc[:,~master_df.columns.duplicated()]
master_df

,subreddit,selftext,title,corrected_title,char_count,word_count,neg,neu,pos,compound,...,zombies,zone,zones,zoo,zoom,zooming,zoos,zorro,zuckerberg,zzquil
0,1,I’ve foreign friends who keep misusing each wo...,what’s the difference between an illness and ...,what’s the difference between an illness and ...,55,9,0.278,0.722,0.000,-0.4019,...,0,0,0,0,0,0,0,0,0,0
1,1,Given all of time why has no one conquered tim...,"if space and time are infinite, why has no on...","if space and time are infinite, why has no on...",59,13,0.278,0.722,0.000,-0.4019,...,0,0,0,0,0,0,0,0,0,0
2,1,[removed],could time travel be possible?,could time travel be possible?,30,5,0.278,0.722,0.000,-0.4019,...,0,0,0,0,0,0,0,0,0,0
3,1,,why do humans need such a wide range of nutri...,why do humans need such a wide range of nutri...,137,22,0.278,0.722,0.000,-0.4019,...,0,0,0,0,0,0,0,0,0,0
4,1,[removed],how does the moon’s orbit and rotation work?,how does the moon’s orbit and rotation work?,44,8,0.278,0.722,0.000,-0.4019,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5655,0,I recently bought bug spray to get rid of spid...,how do i get the smell of anti bug spray out o...,how do i get the smell of anti bug spray out o...,55,14,0.000,0.667,0.333,0.3612,...,0,0,0,0,0,0,0,0,0,0
5656,0,"I was watching a Bollywood movie, where the ma...","if you get stuck between a few bad guys, and t...","if you get stuck between a few bad guys, and t...",110,23,0.000,0.667,0.333,0.3612,...,0,0,0,0,0,0,0,0,0,0
5657,0,[removed],is there anything legally preventing the presi...,is there anything legally preventing the presi...,121,19,0.000,0.667,0.333,0.3612,...,0,0,0,0,0,0,0,0,0,0
5658,0,,why do some people say life is meaningless bec...,why do some people say life is meaningless bec...,286,52,0.000,0.667,0.333,0.3612,...,0,0,0,0,0,0,0,0,0,0


# Model Testing

In [4]:
X = master_df.drop(columns=['subreddit', 'selftext', 'title', 'corrected_title', 'compound', 'pos', 'neu', 'neg'])
y = master_df['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
cat_col = ['corrected_title']
num_col = ['word_count', 'char_count', 'compound', 'pos', 'neu', 'neg']

NameError: name 'master_df' is not defined

In [ ]:
lr = LogisticRegression(max_iter=10_000)
lr.fit(X_train, y_train)
lr.score(X_train, y_train), lr.score(X_test, y_test)
# preds = lr.predict(X_test)
# coef_df = pd.DataFrame(lr.coef_[0], index=cv.get_feature_names_out(), columns=['coef_val'])
# display(coef_df.sort_values(by='coef_val').head(25))
# display(coef_df.sort_values(by='coef_val').tail(25))

In [15]:
y.value_counts(normalize=True)

1    0.503357
0    0.496643
Name: subreddit, dtype: float64

In [20]:
vc1 = VotingClassifier(
    [
        # ('adb', AdaBoostClassifier(random_state=42, n_estimators=500)),
        # ('Boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier(max_features=10, n_estimators=500)),
        ('rfc', RandomForestClassifier(random_state=123, criterion='entropy', min_samples_split=4, n_estimators=500)),
        ('lr', LogisticRegression(max_iter=10_000, C=1, dual=False, fit_intercept=True, intercept_scaling=3, penalty='l2', solver='saga')),
        ('etc', ExtraTreesClassifier(random_state=123, criterion='entropy', min_samples_split=4, n_estimators=500)),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    # ('Boosting', xgb.XGBClassifier()),
    # ('adb', AdaBoostClassifier(random_state=42, n_estimators=500)),
    # ('bagging', BaggingClassifier(max_features=10, n_estimators=500)),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)
# pipe_params = {
#     # 'vc1__lr__max_iter': [10000] 
# }
# gs = GridSearchCV(
#     pipe,
#     pipe_params,
#     n_jobs=-1
# )
# gs.fit(X_train, y_train)
# gs.score(X_train, y_train), gs.score(X_test, y_test)


(0.9992932862190813, 0.7448763250883392)

In [22]:
vc1 = VotingClassifier(
    [
        ('Boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('rfc', RandomForestClassifier()),
        ('lr', LogisticRegression(max_iter=10_000)),
        ('etc', ExtraTreesClassifier()),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9983510011778564, 0.7604240282685513)

In [23]:
vc1 = VotingClassifier(
    [
        ('Boosting', xgb.XGBClassifier()),
        ('adb', AdaBoostClassifier()),
        ('rfc', RandomForestClassifier()),
        ('lr', LogisticRegression(max_iter=10_000)),
        ('etc', ExtraTreesClassifier()),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9818610129564194, 0.7526501766784452)

In [24]:
vc1 = VotingClassifier(
    [
        ('adb', AdaBoostClassifier()),
        ('bagging', BaggingClassifier()),
        ('rfc', RandomForestClassifier()),
        ('lr', LogisticRegression(max_iter=10_000)),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9882214369846879, 0.7590106007067138)

In [25]:
vc1 = VotingClassifier(
    [
        ('boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('rfc', RandomForestClassifier()),
        ('lr', LogisticRegression(max_iter=10_000)),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9936395759717315, 0.7597173144876325)

In [26]:
vc1 = VotingClassifier(
    [
        ('adb', AdaBoostClassifier()),
        ('boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('rfc', RandomForestClassifier()),
        ('lr', LogisticRegression(max_iter=10_000)),
        ('etc', ExtraTreesClassifier()),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9948174322732627, 0.7710247349823321)

In [27]:
vc1 = VotingClassifier(
    [
        ('adb', AdaBoostClassifier()),
        ('boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('rfc', RandomForestClassifier()),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.983510011778563, 0.7865724381625442)

In [29]:
vc1 = VotingClassifier(
    [
        ('adb', AdaBoostClassifier()),
        ('boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('etc', ExtraTreesClassifier()),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.983510011778563, 0.7865724381625442)

In [ ]:
vc1 = VotingClassifier(
    [
        ('adb', AdaBoostClassifier()),
        ('boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('etc', ExtraTreesClassifier()),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

In [ ]:
vc1 = VotingClassifier(
    [
        ('adb', AdaBoostClassifier()),
        ('boosting', xgb.XGBClassifier()),
        ('bagging', BaggingClassifier()),
        ('lr', LogisticRegression(max_iter=10_000)),
    ],
    n_jobs=-1
)
ct = ColumnTransformer([
    ('ss', StandardScaler(), num_col), # polynomial numeric columns
], remainder = 'passthrough')
pipe = Pipeline([
    ('ct', ct),
    ('vc1', vc1)
])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)